In [5]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg

In [6]:
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from datetime import datetime

In [7]:
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')


In [8]:
filename = '2022429_Georgia_Daily_VoterBase/Georgia_Daily_VoterBase.txt'
voter = pd.read_csv(filename, low_memory = False, error_bad_lines=False, sep = '|')
# cleaning up addresses
voter['RESIDENCE_STATE'] = 'GA'
voter['ZIP5'] =  [x[:5] if pd.isna(x) == False else x for x in voter.RESIDENCE_ZIPCODE ]
voter = voter.fillna('')
# strip whitespaces
cols = voter.select_dtypes(object).columns
voter[cols] = voter[cols].apply(lambda x: x.str.strip())

voter['st_address'] = [x + ' ' + y for (x,y) in zip(voter.RESIDENCE_HOUSE_NUMBER , voter.RESIDENCE_STREET_NAME)]
voter['onefield'] = [x + ', ' + y + ', ' + z + ', ' + a 
                         for (x,y,z,a) in 
                         zip(voter.st_address,
                             voter.RESIDENCE_CITY,
                             voter.RESIDENCE_STATE,
                             voter.ZIP5)]

b'Skipping line 4381787: expected 63 fields, saw 114\n'


In [9]:
# the race data is very complete - no null values
voter.RACE.value_counts().sum()/len(voter)

1.0

In [10]:
# but there's also 709314 unknowns and 147733 others, which is about 11%
voter.RACE_DESC.value_counts()

White not of Hispanic Origin         3942041
Black not of Hispanic Origin         2231619
Unknown                               709314
Hispanic                              284592
Asian or Pacific Islander             201779
Other                                 147733
American Indian or Alaskan Native      28235
Name: RACE_DESC, dtype: int64

In [11]:
voter.RACE_DESC.value_counts()/len(voter)*100

White not of Hispanic Origin         52.244897
Black not of Hispanic Origin         29.576228
Unknown                               9.400723
Hispanic                              3.771772
Asian or Pacific Islander             2.674230
Other                                 1.957944
American Indian or Alaskan Native     0.374206
Name: RACE_DESC, dtype: float64

In [12]:
# matching up the geocoding results
addr1 = pd.read_csv('20220503_new_voter_file_addresses.csv', low_memory = False)

In [18]:
# some of the addresses didn't geocode but they didn't show up as null in the spreadsheet until now.
addrnull = addr1[pd.isna(addr1.lat) == True].copy()

In [19]:
addrnull.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392511 entries, 4 to 3201658
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       392511 non-null  int64  
 1   st_address       392511 non-null  object 
 2   RESIDENCE_CITY   390751 non-null  object 
 3   RESIDENCE_STATE  392511 non-null  object 
 4   ZIP5             390889 non-null  object 
 5   onefield         392511 non-null  object 
 6   address          250194 non-null  object 
 7   lat              0 non-null       float64
 8   long             0 non-null       float64
dtypes: float64(2), int64(1), object(6)
memory usage: 29.9+ MB


In [20]:
addr2 = pd.read_excel('c20220504_voter_geocode_arcgis_TableToExcel.xlsx')

In [21]:
addr2_1 = addr2[['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5','Y','X']].copy().drop_duplicates()

In [22]:
addr2_1 = addr2_1.rename(columns={'Y':'lat','X':'long'})

In [23]:
addr2_2 = addr2_1[addr2_1.lat != 0].copy()

In [24]:
addr2_2.ZIP5 = addr2_2.ZIP5.astype('str')

In [25]:
addrnull1 = pd.merge(addrnull,addr2_2, on = ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5'], how = 'left')

In [26]:
arc_round_2 = addrnull1[pd.isna(addrnull1.lat_y) == True][['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']].drop_duplicates()

In [27]:
arc_round_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256830 entries, 0 to 392510
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   st_address       256830 non-null  object
 1   RESIDENCE_CITY   255070 non-null  object
 2   RESIDENCE_STATE  256830 non-null  object
 3   ZIP5             255208 non-null  object
dtypes: object(4)
memory usage: 9.8+ MB


In [28]:
addr1_1 = addr1[pd.isna(addr1.lat) == False].copy()

In [29]:
del addr1

In [30]:
del addr2
del addr2_1

In [31]:
addr3 = pd.read_excel('c20220512_voter_address_r2_TableToExcel.xlsx')

In [32]:
addr3_1 = addr3[['USER_st_address','USER_RESIDENCE_CITY','USER_RESIDENCE_STATE','USER_ZIP5','Y','X']].copy().drop_duplicates()
addr3_1 = addr3_1.rename(columns={'Y':'lat','X':'long'})

In [33]:
addr3_1.columns = addr3_1.columns.str.replace('USER_','')

In [34]:
addr3_1[addr3_1.lat == 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9237 entries, 150 to 256822
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   st_address       9237 non-null   object 
 1   RESIDENCE_CITY   7481 non-null   object 
 2   RESIDENCE_STATE  9237 non-null   object 
 3   ZIP5             9237 non-null   float64
 4   lat              9237 non-null   float64
 5   long             9237 non-null   float64
dtypes: float64(3), object(3)
memory usage: 505.1+ KB


In [35]:
addr3_2 = addr3_1[addr3_1.lat != 0].copy().drop_duplicates()

In [36]:
addr3_2.ZIP5 = np.round(addr3_2.ZIP5,0)

In [37]:
addr3_2.ZIP5 = addr3_2.ZIP5.astype('str')

In [38]:
addr3_2.ZIP5 = [x[:5] for x in addr3_2.ZIP5]

In [39]:
addr3_2.head()

,st_address,RESIDENCE_CITY,RESIDENCE_STATE,ZIP5,lat,long
0,262 HIGHWAY 52,MAYSVILLE,GA,30558,34.261142,-83.575232
1,297 WILBANKS RD,TOCCOA,GA,30577,34.469818,-83.417127
2,223 ED REED RD,LULA,GA,30554,34.380524,-83.544025
3,6068 OLD HIGHWAY 441 N,BALDWIN,GA,30511,34.466205,-83.516968
4,250 SKYVIEW CIR,CLARKESVILLE,GA,30523,34.637657,-83.617786


In [40]:
del addr3

In [41]:
del addr3_1

In [42]:
alladdr = pd.concat([addr1_1,addr2_2,addr3_2])

In [43]:
alladdr = alladdr.drop(['Unnamed: 0','onefield','address'], axis = 1)

In [44]:
for col in alladdr.select_dtypes(include='object').columns:
    alladdr[col] = [str(x) for x in alladdr[col]]
    alladdr[col] = alladdr[col].str.strip()

In [45]:
alladdr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3192425 entries, 0 to 256829
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   st_address       object 
 1   RESIDENCE_CITY   object 
 2   RESIDENCE_STATE  object 
 3   ZIP5             object 
 4   lat              float64
 5   long             float64
dtypes: float64(2), object(4)
memory usage: 170.5+ MB


deconflict address 

In [46]:
checkdup = pd.DataFrame(alladdr.groupby(['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).long.count())

In [51]:
print('number of addresses that have more than one set of coordinates')
len(checkdup[checkdup.long !=1])

number of addresses that have more than one set of coordinates


1534

deconflicting rule - pick the maximum longitude

In [52]:
longmax = pd.DataFrame(alladdr.groupby(['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']).long.max())

In [56]:
len(longmax) - len(alladdr)

-1534

In [57]:
longmax['keep'] = True

In [58]:
longmax = longmax.reset_index()

In [61]:
alladdr1 = pd.merge(alladdr, longmax, on = ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5','long'], how = 'left' )

In [64]:
len(alladdr1[alladdr1.keep == True])- len(alladdr1)

-1534

In [65]:
alladdr2 = alladdr1[alladdr1.keep == True].copy().drop('keep', axis = 1)

In [66]:
alladdr2.head()

,st_address,RESIDENCE_CITY,RESIDENCE_STATE,ZIP5,lat,long
0,237 MITCHELL RD,MAYSVILLE,GA,30558,34.267868,-83.562004
1,467 HICKORY CREEK RD,MAYSVILLE,GA,30558,34.308254,-83.548330
2,733 CARSON SEGARS RD,MAYSVILLE,GA,30558,34.295063,-83.550200
3,11 W RIDGEWAY RD,MAYSVILLE,GA,30558,34.257618,-83.548480
4,61 HOMER ST,MAYSVILLE,GA,30558,34.261340,-83.559350


In [67]:
alladdr = alladdr2.copy()

In [68]:
voterdemo = voter[['REGISTRATION_NUMBER','BIRTHDATE','RACE','st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5']].copy().drop_duplicates()

In [69]:
for col in voterdemo.select_dtypes(include='object').columns:
    voterdemo[col] = voterdemo[col].astype('str')
    voterdemo[col] = voterdemo[col].str.strip()

In [70]:
del voter

In [71]:
voter1 = pd.merge(voterdemo, alladdr, on = ['st_address','RESIDENCE_CITY','RESIDENCE_STATE','ZIP5'], how = 'left')

In [73]:
len(voter1.REGISTRATION_NUMBER.unique())/len(voter1)

1.0

In [74]:
voter1.to_csv('20220516_voter_race_location.csv')